In [16]:
import sys
import os

# Get the absolute path to the project folder, then to 'utilities'
project_path = os.path.abspath("..")  # This goes up one level from 'scrapingFiles'
utilities_path = os.path.join(project_path, "utilities")

# Add 'utilities' folder to Python's path if it's not already there
if utilities_path not in sys.path:
    sys.path.append(utilities_path)
print(utilities_path)

c:\Users\hp\OneDrive\Documents\redbus_scraping\redbusWebscraping\utilities


In [ ]:
# Function to scrape all pages

from commonFunctions import exportCsv, initialize_driver, load_page, scrape_bus_routes
from config import redbusUrl
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL=redbusUrl

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content
            # Find bus item details
            bus_name= driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)): # Total number of bus found on the page
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver,URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

            driver.quit()
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# List to hold all bus details
all_bus_details = []

# Scrape routes and details from all pages
scrape_all_pages()

exportCsv(all_bus_details,'Telangana_bus_details.csv')


Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/kodad-to-hyderabad: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6CF6E38A5+3004357]
	(No symbol) [0x00007FF6CF379970]
	(No symbol) [0x00007FF6CF22582A]
	(No symbol) [0x00007FF6CF275B8E]
	(No symbol) [0x00007FF6CF275E7C]
	(No symbol) [0x00007FF6CF2BEC27]
	(No symbol) [0x00007FF6CF29BC1F]
	(No symbol) [0x00007FF6CF2BBA4C]
	(No symbol) [0x00007FF6CF29B983]
	(No symbol) [0x00007FF6CF267628]
	(No symbol) [0x00007FF6CF268791]
	GetHandleVerifier [0x00007FF6CF70A00D+3161901]
	GetHandleVerifier [0x00007FF6CF75E060+3506048]
	GetHandleVerifier [0x00007FF6CF75400D+3465005]
	GetHandleVerifier [0x00007FF6CF4D0EEB+830987]
	(No symbol) [0x00007FF6CF38467F]
	(No symbol) [0x00007FF6CF3809D4]
	(No symbol) [0x00007FF6CF380B6D]
	(No symbol) [0x00007FF6CF370149]
	BaseThreadInitThunk [0x00007FF8054E7374+20]
	RtlUserThreadStart [0x00007FF8058BCC91+33]

